In [11]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence

# Cargar el dataset desde el archivo Excel
data = pd.read_excel('dataset.xlsx')

device = "cpu"

# Preprocesamiento de los datos
X = data['Oraciones']
y = data['Valor']

# Codificar las etiquetas
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenización y padding de las secuencias
tokenizer = lambda x: x.split()  # Tokenizador simple que separa por espacios
X_train_tokens = [tokenizer(sentence) for sentence in X_train]
X_test_tokens = [tokenizer(sentence) for sentence in X_test]

# Obtener la longitud máxima de las secuencias
max_len = max(len(tokens) for tokens in X_train_tokens)

# Función para convertir tokens a índices
def tokens_to_indices(tokens):
    return torch.tensor([word_to_idx.get(token, 0) for token in tokens], dtype=torch.long)

# Crear un vocabulario a partir de los tokens en el conjunto de entrenamiento
vocab = set(token for tokens in X_train_tokens for token in tokens)
word_to_idx = {word: idx + 1 for idx, word in enumerate(vocab)}  # +1 para dejar 0 para el padding
word_to_idx['<pad>'] = 0  # Agregar token de padding
idx_to_word = {idx: word for word, idx in word_to_idx.items()}

# Convertir tokens a índices y aplicar padding
X_train_indices = [tokens_to_indices(tokens) for tokens in X_train_tokens]
X_test_indices = [tokens_to_indices(tokens) for tokens in X_test_tokens]

# Función para realizar padding de las secuencias
def pad_sequence_to_max_len(sequence, max_len):
    pad_len = max_len - len(sequence)
    return torch.cat([sequence, torch.zeros(pad_len, dtype=torch.long)])

# Aplicar padding a las secuencias
X_train_padded = [pad_sequence_to_max_len(sequence, max_len) for sequence in X_train_indices]
X_test_padded = [pad_sequence_to_max_len(sequence, max_len) for sequence in X_test_indices]

# Convertir a tensores de PyTorch
X_train_tensor = torch.stack(X_train_padded)
X_test_tensor = torch.stack(X_test_padded)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Definir el modelo de CNN
class CNNTextClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_filters, filter_sizes, hidden_dim, output_dim, dropout):
        super(CNNTextClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=embedding_dim, out_channels=num_filters, kernel_size=fs)
            for fs in filter_sizes
        ])
        self.fc = nn.Linear(len(filter_sizes) * num_filters, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        embedded = self.embedding(x)  # [batch_size, seq_len, embedding_dim]
        embedded = embedded.permute(0, 2, 1)  # [batch_size, embedding_dim, seq_len]
        conved = [torch.relu(conv(embedded)) for conv in self.convs]  # conv_n: [batch_size, num_filters, seq_len - filter_sizes[n] + 1]
        pooled = [torch.max(conv, dim=2)[0] for conv in conved]  # pooled_n: [batch_size, num_filters]
        cat = self.dropout(torch.cat(pooled, dim=1))  # [batch_size, num_filters * len(filter_sizes)]
        fc_output = torch.relu(self.fc(cat))  # [batch_size, hidden_dim]
        output = self.output(fc_output)  # [batch_size, output_dim]
        return output

# Parámetros del modelo
vocab_size = len(word_to_idx)
embedding_dim = 100
num_filters = 100
filter_sizes = [3, 4, 5]
hidden_dim = 100
output_dim = len(label_encoder.classes_)
dropout = 0.5

# Inicializar y entrenar el modelo
model = CNNTextClassifier(vocab_size, embedding_dim, num_filters, filter_sizes, hidden_dim, output_dim, dropout)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento del modelo
def train_model(model, optimizer, criterion, X_train, y_train, batch_size, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0
        for i in range(0, len(X_train), batch_size):
            optimizer.zero_grad()
            batch_X = X_train[i:i+batch_size].to(device)
            batch_y = y_train[i:i+batch_size].to(device)
            output = model(batch_X)
            loss = criterion(output, batch_y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(X_train):.4f}')

train_model(model, optimizer, criterion, X_train_tensor, y_train_tensor, batch_size=64, num_epochs=10)

# Evaluación del modelo
def evaluate_model(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        predictions = model(X_test).argmax(dim=1)
        accuracy = accuracy_score(y_test, predictions.cpu().numpy())
        print(f'Accuracy: {accuracy:.4f}')

evaluate_model(model, X_test_tensor, y_test)


Epoch [1/10], Loss: 0.0430
Epoch [2/10], Loss: 0.0214
Epoch [3/10], Loss: 0.0069
Epoch [4/10], Loss: 0.0018
Epoch [5/10], Loss: 0.0005
Epoch [6/10], Loss: 0.0002
Epoch [7/10], Loss: 0.0001
Epoch [8/10], Loss: 0.0001
Epoch [9/10], Loss: 0.0001
Epoch [10/10], Loss: 0.0000
Accuracy: 1.0000


In [24]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Cargar el dataset desde el archivo Excel
data = pd.read_excel('dataset.xlsx')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Preprocesamiento de los datos
X = data['Oraciones']
y = data['Valor']

# Codificar las etiquetas
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenización y padding de las secuencias
tokenizer = lambda x: x.split()  # Tokenizador simple que separa por espacios
X_train_tokens = [tokenizer(sentence) for sentence in X_train]
X_test_tokens = [tokenizer(sentence) for sentence in X_test]

# Obtener la longitud máxima de las secuencias
max_len = max(len(tokens) for tokens in X_train_tokens)

# Crear un vocabulario a partir de los tokens en el conjunto de entrenamiento
vocab = set(token for tokens in X_train_tokens for token in tokens)
word_to_idx = {word: idx + 1 for idx, word in enumerate(vocab)}  # +1 para dejar 0 para el padding
word_to_idx['<pad>'] = 0  # Agregar token de padding
idx_to_word = {idx: word for word, idx in word_to_idx.items()}

# Convertir tokens a índices y aplicar padding
X_train_indices = [[word_to_idx.get(token, 0) for token in tokens] for tokens in X_train_tokens]
X_test_indices = [[word_to_idx.get(token, 0) for token in tokens] for tokens in X_test_tokens]

# Aplicar padding a las secuencias
X_train_padded = [sequence + [0] * (max_len - len(sequence)) for sequence in X_train_indices]
X_test_padded = [sequence + [0] * (max_len - len(sequence)) for sequence in X_test_indices]

# Convertir a tensores de PyTorch y mover a GPU si está disponible
X_train_tensor = torch.tensor(X_train_padded, dtype=torch.long, device=device)
X_test_tensor = torch.tensor(X_test_padded, dtype=torch.long, device=device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long, device=device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long, device=device)

# Definir el modelo de CNN
class CNNTextClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_filters, filter_sizes, hidden_dim, output_dim, dropout):
        super(CNNTextClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=embedding_dim, out_channels=num_filters, kernel_size=fs)
            for fs in filter_sizes
        ])
        self.fc = nn.Linear(len(filter_sizes) * num_filters, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        embedded = self.embedding(x)  # [batch_size, seq_len, embedding_dim]
        embedded = embedded.permute(0, 2, 1)  # [batch_size, embedding_dim, seq_len]
        conved = [torch.relu(conv(embedded)) for conv in self.convs]  # conv_n: [batch_size, num_filters, seq_len - filter_sizes[n] + 1]
        pooled = [torch.max(conv, dim=2)[0] for conv in conved]  # pooled_n: [batch_size, num_filters]
        cat = self.dropout(torch.cat(pooled, dim=1))  # [batch_size, num_filters * len(filter_sizes)]
        fc_output = torch.relu(self.fc(cat))  # [batch_size, hidden_dim]
        output = self.output(fc_output)  # [batch_size, output_dim]
        return output

# Parámetros del modelo
vocab_size = len(word_to_idx)
embedding_dim = 100
num_filters = 100
filter_sizes = [3, 4, 5]
hidden_dim = 100
output_dim = len(label_encoder.classes_)
dropout = 0.5

# Inicializar y entrenar el modelo en CUDA
model = CNNTextClassifier(vocab_size, embedding_dim, num_filters, filter_sizes, hidden_dim, output_dim, dropout).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento del modelo
def train_model(model, optimizer, criterion, X_train, y_train, batch_size, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0
        for i in range(0, len(X_train), batch_size):
            optimizer.zero_grad()
            batch_X = X_train[i:i+batch_size]
            batch_y = y_train[i:i+batch_size]
            output = model(batch_X)
            loss = criterion(output, batch_y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(X_train):.4f}')

train_model(model, optimizer, criterion, X_train_tensor, y_train_tensor, batch_size=64, num_epochs=10)

# Evaluación del modelo
def evaluate_model(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        predictions = model(X_test).argmax(dim=1)
        accuracy = accuracy_score(y_test.cpu().numpy(), predictions.cpu().numpy())
        print(f'Accuracy: {accuracy:.4f}')

evaluate_model(model, X_test_tensor, y_test_tensor)


Epoch [1/10], Loss: 0.0449
Epoch [2/10], Loss: 0.0248
Epoch [3/10], Loss: 0.0089
Epoch [4/10], Loss: 0.0022
Epoch [5/10], Loss: 0.0006
Epoch [6/10], Loss: 0.0002
Epoch [7/10], Loss: 0.0001
Epoch [8/10], Loss: 0.0001
Epoch [9/10], Loss: 0.0001
Epoch [10/10], Loss: 0.0001
Accuracy: 1.0000


In [19]:
# Cargar el dataset desde el archivo Excel
data = pd.read_excel('dataset.xlsx')
print(data)
print("############################################################################")
data

                                             Oraciones                 Valor
0    Instalación pendiente por falta de permisos mu...  Permisos municipales
1    Esperando asignación de recurso técnico para r...       Recurso técnico
2    Falla reportada en el servicio de internet, có...     Reporte de fallas
3    Cambio de domicilio solicitado, nueva direcció...   Cambio de domicilio
4    Servicio suspendido temporalmente por falta de...   Suspensión temporal
..                                                 ...                   ...
495  Esperando asignación de recurso técnico para r...       Recurso técnico
496  Falla reportada en el servicio de internet, có...     Reporte de fallas
497  Cambio de domicilio solicitado, nueva direcció...   Cambio de domicilio
498  Servicio suspendido temporalmente por falta de...   Suspensión temporal
499  Agendado en ETAdvanced versión 2.3, fecha: 15/...            ETAdvanced

[500 rows x 2 columns]
####################################################

,Oraciones,Valor
0,Instalación pendiente por falta de permisos mu...,Permisos municipales
1,Esperando asignación de recurso técnico para r...,Recurso técnico
2,"Falla reportada en el servicio de internet, có...",Reporte de fallas
3,"Cambio de domicilio solicitado, nueva direcció...",Cambio de domicilio
4,Servicio suspendido temporalmente por falta de...,Suspensión temporal
...,...,...
495,Esperando asignación de recurso técnico para r...,Recurso técnico
496,"Falla reportada en el servicio de internet, có...",Reporte de fallas
497,"Cambio de domicilio solicitado, nueva direcció...",Cambio de domicilio
498,Servicio suspendido temporalmente por falta de...,Suspensión temporal


No charts were generated by quickchart


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_1.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_2['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

In [21]:
def predict_sentence(sentence):
    # Tokenizar la oración
    tokens = tokenizer(sentence)
    # Convertir tokens a índices y aplicar padding
    indices = [word_to_idx.get(token, 0) for token in tokens]
    padded_indices = indices + [0] * (max_len - len(indices))
    # Convertir a tensor de PyTorch y mover a la GPU si está disponible
    tensor = torch.tensor(padded_indices, dtype=torch.long, device=device).unsqueeze(0)
    # Pasar la oración por el modelo
    model.eval()
    with torch.no_grad():
        output = model(tensor).argmax(dim=1)
        predicted_label = label_encoder.inverse_transform(output.cpu().numpy())[0]
    return predicted_label

# Ejemplo de uso:
sentence = "Instalación pendiente por falta de permisos globales"
predicted_label = predict_sentence(sentence)
print(f'La oración "{sentence}" pertenece a la categoría "{predicted_label}".')


La oración "Instalación pendiente por falta de permisos globales" pertenece a la categoría "Permisos municipales".


In [33]:
import torch
from sklearn.preprocessing import LabelEncoder
import pickle

# Guardar el modelo entrenado
torch.save(model, 'modelo_entrenado.pth')

# Definir una función separada para el tokenizador
def tokenizer(text):
    return text.split()

# Guardar la metadata en un archivo pickle
metadata = {
    'tokenizer': tokenizer,
    'word_to_idx': word_to_idx,
    'max_len': max_len,
    'label_encoder': label_encoder
}
with open('metadata.pkl', 'wb') as f:
    pickle.dump(metadata, f)

# Función para cargar el modelo y la metadata
def load_model_and_metadata(model_path, metadata_path):
    # Cargar el modelo
    model = torch.load(model_path)

    # Cargar la metadata desde el archivo pickle
    with open(metadata_path, 'rb') as f:
        metadata = pickle.load(f)
        tokenizer = metadata['tokenizer']
        word_to_idx = metadata['word_to_idx']
        max_len = metadata['max_len']
        label_encoder = metadata['label_encoder']

    return model, tokenizer, word_to_idx, max_len, label_encoder

# Ejemplo de cómo cargar el modelo y la metadata
model_path = 'modelo_entrenado.pth'
metadata_path = 'metadata.pkl'
model, tokenizer, word_to_idx, max_len, label_encoder = load_model_and_metadata(model_path, metadata_path)
print(f"Se guardo la metadata:{metadata_path} y el modelo:{model_path}")

Se guardo la metadata:metadata.pkl y el modelo:modelo_entrenado.pth


In [45]:
import torch
import pickle

# Función para cargar el modelo y la metadata
def load_model_and_metadata(model_path, metadata_path):
    # Cargar el modelo
    model = torch.load(model_path, map_location=torch.device('cpu'))

    # Cargar la metadata desde el archivo pickle
    with open(metadata_path, 'rb') as f:
        metadata = pickle.load(f)
        tokenizer = metadata['tokenizer']
        word_to_idx = metadata['word_to_idx']
        max_len = metadata['max_len']
        label_encoder = metadata['label_encoder']

    return model, tokenizer, word_to_idx, max_len, label_encoder

# Cargar el modelo y la metadata
model_path = 'modelo_entrenado.pth'
metadata_path = 'metadata.pkl'
model, tokenizer, word_to_idx, max_len, label_encoder = load_model_and_metadata(model_path, metadata_path)

# Función para realizar predicciones con el modelo cargado
def predict(sentence, model, tokenizer, word_to_idx, max_len, label_encoder):
    # Tokenizar la oración
    tokens = tokenizer(sentence)
    # Convertir tokens a índices
    indices = [word_to_idx.get(token, 0) for token in tokens]
    # Aplicar padding si es necesario
    if len(indices) < max_len:
        indices += [0] * (max_len - len(indices))
    elif len(indices) > max_len:
        indices = indices[:max_len]
    # Convertir a tensor y agregar dimensión de lote
    input_tensor = torch.tensor([indices], dtype=torch.long)
    # Mover el tensor al mismo dispositivo que el modelo
    input_tensor = input_tensor.to(torch.device('cpu'))
    # Realizar predicción
    with torch.no_grad():
        output = model(input_tensor)
        predicted_class_idx = torch.argmax(output).item()
    # Convertir el índice de clase predicho a la etiqueta correspondiente
    predicted_class = label_encoder.inverse_transform([predicted_class_idx])[0]
    return predicted_class

# Ejemplo de uso
sentence = "Instalación pendiente por falta de permisos municipales"
predicted_class = predict(sentence, model, tokenizer, word_to_idx, max_len, label_encoder)
print("Predicted class:", predicted_class)


Predicted class: Permisos municipales


# Entrenar con los datos reales

In [47]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import pickle

# Función de tokenización
def simple_tokenizer(sentence):
    return sentence.split()

# Cargar el dataset desde el archivo Excel
data = pd.read_excel('dataset.xlsx')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Preprocesamiento de los datos
X = data['Oraciones']
y = data['Valor']

# Codificar las etiquetas
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenización y padding de las secuencias
tokenizer = simple_tokenizer  # Utilizar la función de tokenización definida
X_train_tokens = [tokenizer(sentence) for sentence in X_train]
X_test_tokens = [tokenizer(sentence) for sentence in X_test]

# Obtener la longitud máxima de las secuencias
max_len = max(len(tokens) for tokens in X_train_tokens)

# Crear un vocabulario a partir de los tokens en el conjunto de entrenamiento
vocab = set(token for tokens in X_train_tokens for token in tokens)
word_to_idx = {word: idx + 1 for idx, word in enumerate(vocab)}  # +1 para dejar 0 para el padding
word_to_idx['<pad>'] = 0  # Agregar token de padding
idx_to_word = {idx: word for word, idx in word_to_idx.items()}

# Convertir tokens a índices y aplicar padding
X_train_indices = [[word_to_idx.get(token, 0) for token in tokens] for tokens in X_train_tokens]
X_test_indices = [[word_to_idx.get(token, 0) for token in tokens] for tokens in X_test_tokens]

# Aplicar padding a las secuencias
X_train_padded = [sequence + [0] * (max_len - len(sequence)) for sequence in X_train_indices]
X_test_padded = [sequence + [0] * (max_len - len(sequence)) for sequence in X_test_indices]

# Convertir a tensores de PyTorch y mover a GPU si está disponible
X_train_tensor = torch.tensor(X_train_padded, dtype=torch.long, device=device)
X_test_tensor = torch.tensor(X_test_padded, dtype=torch.long, device=device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long, device=device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long, device=device)

# Definir el modelo de CNN
class CNNTextClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_filters, filter_sizes, hidden_dim, output_dim, dropout):
        super(CNNTextClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=embedding_dim, out_channels=num_filters, kernel_size=fs)
            for fs in filter_sizes
        ])
        self.fc = nn.Linear(len(filter_sizes) * num_filters, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        embedded = self.embedding(x)  # [batch_size, seq_len, embedding_dim]
        embedded = embedded.permute(0, 2, 1)  # [batch_size, embedding_dim, seq_len]
        conved = [torch.relu(conv(embedded)) for conv in self.convs]  # conv_n: [batch_size, num_filters, seq_len - filter_sizes[n] + 1]
        pooled = [torch.max(conv, dim=2)[0] for conv in conved]  # pooled_n: [batch_size, num_filters]
        cat = self.dropout(torch.cat(pooled, dim=1))  # [batch_size, num_filters * len(filter_sizes)]
        fc_output = torch.relu(self.fc(cat))  # [batch_size, hidden_dim]
        output = self.output(fc_output)  # [batch_size, output_dim]
        return output

# Parámetros del modelo
vocab_size = len(word_to_idx)
embedding_dim = 100
num_filters = 100
filter_sizes = [3, 4, 5]
hidden_dim = 100
output_dim = len(label_encoder.classes_)
dropout = 0.5

# Inicializar y entrenar el modelo en CUDA
model = CNNTextClassifier(vocab_size, embedding_dim, num_filters, filter_sizes, hidden_dim, output_dim, dropout).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento del modelo
def train_model(model, optimizer, criterion, X_train, y_train, batch_size, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0
        for i in range(0, len(X_train), batch_size):
            optimizer.zero_grad()
            batch_X = X_train[i:i+batch_size]
            batch_y = y_train[i:i+batch_size]
            output = model(batch_X)
            loss = criterion(output, batch_y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(X_train):.4f}')

train_model(model, optimizer, criterion, X_train_tensor, y_train_tensor, batch_size=64, num_epochs=10)

# Evaluación del modelo
def evaluate_model(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        predictions = model(X_test).argmax(dim=1)
        accuracy = accuracy_score(y_test.cpu().numpy(), predictions.cpu().numpy())
        print(f'Accuracy: {accuracy:.4f}')

evaluate_model(model, X_test_tensor, y_test_tensor)

# Guardar el modelo y la metadata
metadata = {
    'tokenizer': simple_tokenizer,
    'word_to_idx': word_to_idx,
    'max_len': max_len,
    'label_encoder': label_encoder
}
with open('metadata.pkl', 'wb') as f:
    pickle.dump(metadata, f)


Epoch [1/10], Loss: 0.0438
Epoch [2/10], Loss: 0.0236
Epoch [3/10], Loss: 0.0089
Epoch [4/10], Loss: 0.0022
Epoch [5/10], Loss: 0.0006
Epoch [6/10], Loss: 0.0002
Epoch [7/10], Loss: 0.0001
Epoch [8/10], Loss: 0.0001
Epoch [9/10], Loss: 0.0001
Epoch [10/10], Loss: 0.0000
Accuracy: 1.0000


In [49]:
import torch
import pickle

# Definir la función de tokenización
def simple_tokenizer(sentence):
    return sentence.split()

# Función para cargar el modelo y la metadata
def load_model_and_metadata(model_path, metadata_path):
    # Cargar el modelo
    model = torch.load(model_path, map_location=torch.device('cpu'))

    # Cargar la metadata desde el archivo pickle
    with open(metadata_path, 'rb') as f:
        metadata = pickle.load(f)
        tokenizer = metadata['tokenizer']
        word_to_idx = metadata['word_to_idx']
        max_len = metadata['max_len']
        label_encoder = metadata['label_encoder']

    return model, tokenizer, word_to_idx, max_len, label_encoder

# Cargar el modelo y la metadata
model_path = 'modelo_entrenado.pth'
metadata_path = 'metadata.pkl'
model, tokenizer, word_to_idx, max_len, label_encoder = load_model_and_metadata(model_path, metadata_path)

# Función para realizar predicciones con el modelo cargado
def predict(sentence, model, tokenizer, word_to_idx, max_len, label_encoder):
    # Tokenizar la oración
    tokens = tokenizer(sentence)
    # Convertir tokens a índices
    indices = [word_to_idx.get(token, 0) for token in tokens]
    # Aplicar padding si es necesario
    if len(indices) < max_len:
        indices += [0] * (max_len - len(indices))
    elif len(indices) > max_len:
        indices = indices[:max_len]
    # Convertir a tensor y agregar dimensión de lote
    input_tensor = torch.tensor([indices], dtype=torch.long)
    # Mover el tensor al mismo dispositivo que el modelo
    input_tensor = input_tensor.to(torch.device('cpu'))
    # Realizar predicción
    with torch.no_grad():
        output = model(input_tensor)
        predicted_class_idx = torch.argmax(output).item()
    # Convertir el índice de clase predicho a la etiqueta correspondiente
    predicted_class = label_encoder.inverse_transform([predicted_class_idx])[0]
    return predicted_class

# Ejemplo de uso
sentence = "Instalación pendiente por falta de permisos municipales"
predicted_class = predict(sentence, model, tokenizer, word_to_idx, max_len, label_encoder)
print("Predicted class:", predicted_class)


Predicted class: Permisos municipales
